In [11]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [12]:
#Jennifer Sleeman 
#DC Data Science Meetup
#Example of DCGAN in Keras
#Based on code from the following sources:
#https://github.com/jacobgil/keras-dcgan/,https://github.com/rajathkumarmp/DCGAN/
import pylab as pl
from IPython import display
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten
from keras.optimizers import SGD,Adam
from keras.datasets import mnist
import numpy as np
import argparse
import math


In [13]:
def getMNIST():
    #Get the data
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    #normalize
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    #Reshape
    X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
    return (X_train,y_train,X_test,y_test)

#plot the loss
def plot_loss(dloss,gloss):
    from IPython import display
    display.clear_output(wait=True)
    display.display(plt.gcf())
    plot(dloss[0:],label='discriminitive loss')
    plot(gloss, label='generative loss')
    legend()
    show()
    
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(np.sqrt(num))
    height = int(np.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]), dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = img[0, :, :]
    return image

#Save the generated images to the file system
def save_generated(generated_dir, epoch, index,generated_images):
    for index2,img in enumerate(generated_images):
        image = combine_images(generated_images)
        #image=img[0,:,:]
        image=image*127.5+127.5
    from PIL import Image
    Image.fromarray((image).astype(np.uint8)).save(generated_dir+str(epoch)+"_"+str(index)+".png")
    
#Assign random noise
#Draw samples from a uniform distribution
def assign_random_noise(batch_size, noise):
       for i in range(batch_size):
            noise[i, :] = np.random.uniform(-1, 1, 100)
            
#Save the models for later
def save_models(discriminator,generator):            
    generator.save_weights('generator', True)
    discriminator.save_weights('discriminator', True)   
    
#Print the losses, plot the losses and print one the images
def print_stats(g_loss,d_loss,generated_dir,epoch,index):
    #save the weights periodically
    plot_loss(d_loss, g_loss)
    from IPython.display import Image,display
    display(Image(filename=generated_dir+str(epoch)+"_"+str(index)+".png"))
    print("batch "+ str(index) +" d_loss :"+  str(d_loss[len(d_loss)-1]))
    print("batch "+ str(index) +" g_loss : "+ str(g_loss[len(g_loss)-1]))

In [14]:
#Create the generator model
def generator_model():
    #linear stack of layers model
    generator = Sequential()
    #take in 100 random inputs
    generator.add(Dense( input_dim=100, output_dim=(128*7*7)))
    #ramp function 0,inf
    generator.add(Activation('relu'))
    #reshaping to 128, 7,7
    generator.add(Reshape((128, 7, 7)))
    #upsampling repeat rows and columns by 2,2
    generator.add(UpSampling2D(size=(2, 2)))
    #128, 14,14
    generator.add(Convolution2D(64, 5, 5, border_mode='same'))
    #ramp function 0,inf
    generator.add(Activation('relu'))
    #upsampling repeat rows and columns by 2,2
    generator.add(UpSampling2D(size=(2, 2)))
    #128,28,28
    generator.add(Convolution2D(1, 5, 5, border_mode='same'))
    #tanh -1,1
    generator.add(Activation('tanh'))  
    return generator

In [15]:
#Create the discriminative model
def discriminator_model():
    #linear stack of layers model
    discriminator = Sequential()
    #Convolutional layer with 64 filters and 5 rows 5 columns in kernel, strides 2,2 input shape 1,28,28 for mnist
    discriminator.add(Convolution2D(64, 5, 5, border_mode='same', 
                                    subsample=(2,2), input_shape=(1,28,28)))
    #LeakyReLu activation
    discriminator.add(LeakyReLU(0.2))
    #Convolutional layer with 128 filters and 5 rows 5 columns in kernel, strides 2,2
    discriminator.add(Convolution2D(128, 5, 5, border_mode='same', subsample=(2,2)))
    #LeakyReLu activation
    discriminator.add(LeakyReLU(0.2))
    #Flattens
    discriminator.add(Flatten())
    #Output of dim 1
    discriminator.add(Dense(1))
    #Activation of Sigmoid [0,1]
    discriminator.add(Activation('sigmoid'))
    return discriminator

In [16]:
#We do this so the weights from the discriminator can be used by the generator
def generator_discriminator(generator, discriminator):
    #linear stack of layers model
    model = Sequential()
    #add the generator
    model.add(generator)
    #tell the discriminator freeze its weights
    discriminator.trainable = False
    #add the discriminator
    model.add(discriminator)
    return model

In [17]:
def compile_models(batch_size,learning_rate):
    #define our discriminator
    discriminator = discriminator_model()
    #define our generator
    generator = generator_model()

    discriminator_generator = \
    generator_discriminator(generator, discriminator)

    #set up our parameters (learning rate etc.)
    d_optim = Adam(lr=learning_rate, beta_1=0.5, decay=0.001)
    g_optim = Adam(lr=learning_rate, beta_1=0.5, decay=0.001)

    #compile the generator
    generator.compile(loss='binary_crossentropy', optimizer=g_optim, metrics=['accuracy'])

    #compile the discriminator/generator
    discriminator_generator.compile(
        loss='binary_crossentropy', optimizer=g_optim)
    #compile the discriminator
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim, metrics=['accuracy'])
    return (discriminator,generator,discriminator_generator)
    
    

In [18]:
def train(learning_rate,batch_size,num_examples,num_batches,X_train,Y_train,X_test,Y_test,generated_dir):
    #loop by epoch and go through the training process
    dLoss=[]
    gLoss=[]
    #compile models
    discriminator, generator,discriminator_generator=compile_models(batch_size,learning_rate)
    #setting up the noise # 100 is the contributing to the bottleneck
    noise = np.zeros((batch_size, 100))
    for epoch in range(100):
        #from 0 to number of batches
        for index in range(num_batches):
            #assigning random noise
            assign_random_noise(batch_size, noise)

            #get a batch of images
            image_batch = X_train[index*batch_size:(index+1)*batch_size]

            #get our generated images
            generated_images = generator.predict(noise, verbose=0)

            #periodically save the generated images       
            if index % 30 == 0 or index==0:
                save_generated(generated_dir, epoch, index, generated_images)

            #combine the real and fake images 
            X = np.concatenate((image_batch, generated_images))
            y = [1] * batch_size + [0] * batch_size


            #give the real and fake images to the discriminator
            #Single gradient update over one batch of samples
            d_loss = discriminator.train_on_batch(X, y)
            dLoss.append(d_loss[0])


            #assigning random noise
            assign_random_noise(batch_size, noise)
            discriminator.trainable = False

            #train combined model
            #NOTICE THE LABELS GOING IN ARE ALL 1
            g_loss = discriminator_generator.train_on_batch(noise, [1] * batch_size)
            gLoss.append(g_loss)
            
            #unfreeze 
            discriminator.trainable = True
            #save models
            if index % 10 == 9 or index==0: 
                save_models(discriminator,generator)
                
            #print stats
            if index % 30 == 0:
                print_stats(gLoss,dLoss,generated_dir,epoch,index)
        

In [ ]:
#initialize parameter
#batch_size 128
batch_size = 128
#learning rate
learning_rate = 0.0002
#number of epochs
nepoch = 50

#get the MNIST data
X_train,Y_train,X_test,Y_test=getMNIST()
#number of examples
num_examples = (X_train.shape)[0]
#number of batches
num_batches = int(num_examples/float(batch_size))

print('Number of examples: ',  num_examples)
print('Number of batches: ', num_batches)
print('Number of epochs: ', nepoch)

generated_dir="/home/jsleeman/dev/learn/gans/generated/mnist/"
#generated_dir="/home/ubuntu/generated/mnist/"
train(learning_rate,batch_size,num_examples,num_batches,X_train,Y_train,X_test,Y_test,generated_dir)